In [25]:
""" Artificial Bee Colony Algorithm

This functions runs the Artificial Bee Colony Algorithm with as output the optimal solution of the size D.

Input
- D: number of decision variables
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 
- S: swarm size
- T: number of cycles
- limit: decides when scouts phase needs to be executed (often taken Np*D)
- f: the function that you want to use for computing objective values



Output
- optimal_solution: gives a vector of the size of D with the optimal solution  

"""

function ArtificialBeeColonization(D, bounds_lower, bounds_upper, S, T, limit, f::Function)
    @assert D > 0 # only a positive number of decision variables
    @assert bounds_lower <= bounds_upper # lower bounds must be lower than the upperbounds or equal
    @assert length(bounds_lower) == length(bounds_upper) == D  # length of the boundries must be equal to the number of decision variables
    @assert iseven(S) # swarm size must be an even number
    @assert S > 0 # swarm size can not be negative
    @assert T > 0 # number of cylces must be postive
 
    
    Np = Int8(S/2) # number of food sources/employed bees/onlooker bees
    
    # initialize population
    population = initialize_population(D, bounds_lower, bounds_upper, Np)
    
    # calculate objective values and fitness values for population
    objective_values = compute_objective(population, f)
    fitness_values = compute_fitness(objective_values)
    
    # initialize trial vector for population
    trial = zeros(Np, 1)
    best_fitness = 0
    optimal_solution = []
    populations = []
    for iterations in 1:T
    
        ## EMPLOYED BEE PHASE
        population, fitness_values, objective_values, trial = employed_bee_phase(population, bounds_lower, bounds_upper, trial, Np, f::Function)
    
    
        ## ONLOOKER BEE PHASE
        population, fitness_values, objective_values, trial = onlooker_bee_phase(population, bounds_lower, bounds_upper, trial, Np, f::Function)  
       
        ## SCOUTING PHASE
        if maximum(fitness_values) > best_fitness
            best_fitness = maximum(fitness_values)
            ind = argmax(fitness_values)
            optimal_solution = population[ind]
            
        end
            
        population, fitness_values, objective_values, trial = Scouting(population, bounds_lower, bounds_upper, trial, fitness_values, objective_values, limit, f::Function)
        
        if maximum(fitness_values) > best_fitness
            best_fitness = maximum(fitness_values)
            ind = argmax(fitness_values)
            optimal_solution = population[ind]
            
        end
        populations = append!(populations, [population])
    end

    return optimal_solution,populations
end


ArtificialBeeColonization (generic function with 1 method)

In [53]:
S = 24
D = 2
bounds_lower = [-100,-100];
bounds_upper = [100,100];
limit = D * (S/2)
T= 500


500

In [56]:
optimal_solution,populations = ArtificialBeeColonization(D, bounds_lower, bounds_upper, S, T, limit, rastrigine)
optimal_solution 
# sphere: the minimum value of zero is at (0,0)
# ackley: the minimum value of zero is at (0,0)
# rosebrock: the minimum value of zero is at (1,1)
# brunin: Branin-Hoo, function has three global minima: at (-3.14, 12.275), (3.14, 2.275), (9.42, 2.475)
# rastrigine: the minimum value of zero is at (0,0)

2-element Array{Float64,1}:
 -4.2291042406308465e-10
 -9.508071096677229e-10

In [4]:
""" Initialize population

This function generates Np random solutions (food sources) within the domain 
of the variables to form an initial population for the ABC algorithm.

Input
- D: number of decision variables
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 
- Np: number of food sources/employed bees/onlooker bees

Output 
- population: a random solution of the size D
"""

function initialize_population(D, bounds_lower, bounds_upper, Np)
    population = []   
    for i in 1:Np
        food_source = collect(rand(bounds_lower[i]:bounds_upper[i]) for i in 1:D)
        append!(population, [food_source])
    end
        
    return population
end	




initialize_population (generic function with 1 method)

In [5]:
"""Objective function

Calculates the objective values for a certain function 
Calculation for 1 vector or n instances (vectors) in population.

Input
- input: input values
- f: the function that you want to use for computing objective values

"""
function compute_objective(input, f::Function)
    if length(input)==1
        objective = f(sum(input))
        output = objective
    else
        objectives_population = []
        
        for j in 1:length(input)
            food_source = input[j]
            objective = f(food_source)
            append!(objectives_population, objective)
        end
        
        output = objectives_population
    end
    
    return output
end

compute_objective

In [6]:
""" Fitness function

This functions calculates the fitness of a population.
The fitness is calculated as 1/(1+objective_values).
The bigger the objective values the smaller the fitness values.

Input
- objective values

Output
- fitness values


"""

function compute_fitness(objective_values)
    fitness_values = []
    
    for i in 1:length(objective_values)
        objective_value = objective_values[i]
        
        if objective_value >= 0
            fitness = 1/(1+objective_value)
     
        else
            fitness = 1+abs(objective_value)
        end
        
        append!(fitness_values, fitness)
    end
    return fitness_values
end	



compute_fitness (generic function with 1 method)

In [7]:
""" Employed bee phase function
This functions employs the employed bee phase. 

Input
- population: population of solutions 
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 
- trial: current trial of solutions
- Np: number of food sources/employed bees/onlooker bees
- f: the function that you want to use for computing objective values

Output
- population_new_evolved: new population values
- fitness_new_evolved: new fitness values
- objective_new_evolved: new objective values
- trial: updated trials of solutions in population
    When original solution has failed to generate better solution, trial counter is increased by 1 unit
    When better solution has been found, the trial counter for this new solution is set to zero

"""

function employed_bee_phase(population, bounds_lower, bounds_upper, trial, Np, f::Function)
    population_new = []
    
    # create new food sources
    for i in 1:Np
        solution = population[i, :][1]
        solution_new = solution
        while solution_new == solution
            solution_new = create_newsolution(solution, population, bounds_lower, bounds_upper)
        end
        append!(population_new, [solution_new])
    end
    
    # evaluate fitness old and new population
    objective_values_old = compute_objective(population, f)
    fitness_old = compute_fitness(objective_values_old)
    objective_values_new = compute_objective(population_new, f)
    fitness_new = compute_fitness(objective_values_new)

    # perform greedy selection
    population_new_evolved = []
    fitness_new_evolved = []
    objective_new_evolved = []
    
    for j in 1:Np
        if fitness_new[j] > fitness_old[j]
            append!(population_new_evolved, [population_new[j]])
            append!(fitness_new_evolved, fitness_new[j])
            append!(objective_new_evolved, objective_values_new[j])
            trial[j] = 0
        else 
            append!(population_new_evolved, [population[j]]) 
            append!(fitness_new_evolved, fitness_old[j])
            append!(objective_new_evolved, objective_values_old[j])
            trial[j] += 1
        end
    end
    
    return population_new_evolved, fitness_new_evolved, objective_new_evolved, trial
end



employed_bee_phase (generic function with 1 method)

In [8]:
""" Food source information (measured in probabilities)
This function measures the food source information in probabilities. 
The food source information is calculated as following: 0.9*(fitness_value/maximum(fitness_values)) + 0.1

Input
- fitness_values: fitness values

Output
- probabilities: probabilities of the food source information 


"""

function foodsource_info_prob(fitness_values)
    probabilities = []
    
    for i in 1:length(fitness_values)
        fitness_value = fitness_values[i] 
        probability = 0.9*(fitness_value/maximum(fitness_values)) + 0.1
        append!(probabilities, probability)
    end
    
    return probabilities
end	



foodsource_info_prob (generic function with 1 method)

In [9]:
""" Onlooker bee phase function
This function employs the onlooker bee phase. 

Input
- population: population of solutions 
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 
- trial: current trial of solutions
- Np: number of food sources/employed bees/onlooker bees
- f: the function that you want to use for computing objective values

Output
- population: new population values
- fitness_new_evolved: new fitness values
- objective_new_evolved: new objective values
- trial: updated trials of solutions in population
    When original solution has failed to generate better solution, trial counter is increased by 1 unit
    When better solution has been found, the trial counter for this new solution is set to zero

"""

function onlooker_bee_phase(population, bounds_lower, bounds_upper, trial, Np, f::Function)
    m = 0 # onlooker bee
    n = 1 # food source
    
    objective_values = compute_objective(population,f)
    fitness = compute_fitness(objective_values)
    # first calculate the probability values
    proba = foodsource_info_prob(fitness)
    
    while m <= Np # we want for every onlooker bee a new solution
        r = rand()
        if r <= proba[n]
            solution = population[n, :][1] # solution n
            objective_values_old = compute_objective([solution], f)
            fitness_old = compute_fitness(objective_values_old)
            
            solution_new = solution
            while solution_new == solution
                solution_new = create_newsolution(solution, population, bounds_lower, bounds_upper)
            end
    
            objective_values_new = compute_objective([solution_new], f)
            fitness_new = compute_fitness(objective_values_new)
            
            if fitness_new > fitness_old # if this get accepted 
                population[n, :] = [solution_new]
                trial[n]=0
            else 
                trial[n] += 1
            end
            m = m + 1
        end
        # if the rand < proba is not sattisfied
        n = n +1
        if n > Np 
            n = 1
        end
    end
    objective_new_evolved = compute_objective(population,f)
    fitness_new_evolved = compute_fitness(objective_new_evolved)
    
    return population, fitness_new_evolved, objective_new_evolved, trial
end	




onlooker_bee_phase (generic function with 1 method)

In [10]:
""" Scouting function
This function employs the scouting phase. 

Input
- population : population of solutions 
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 
- trials: current trial of solutions
- fitness: fitness values
- objective: objective values
- limit: limit value
- f: the function that you want to use for computing objective values

Output 
- population: new population values
- fitness: new fitness values
- objective: new objective values
- trials: updated trials of solutions in population
    When original solution has failed to generate better solution, trial counter is increased by 1 unit
    When better solution has been found, the trial counter for this new solution is set to zero


"""

function Scouting(population, bounds_lower, bounds_upper, trials, fitness, objective, limit, f::Function)
        
        # check whether the trial vector exceed the limit value and importantly where
        index_exceed = trials .> limit
    
        if sum(index_exceed) >= 1 # there is minimal one case where we exceed the limit
            if sum(maximum(trials) .== trials) > 1 # multiple cases have the same maximum so chose randomly
                possible_scoutings = findall(trials .== maximum(trials))
                idx = rand(1:size(possible_scoutings)[1])
                global scouting_array = possible_scoutings[idx]
            else # only one array has a maximum => chose this one 
            
                global scouting_array = argmax(trials)
            end
            pop = population[scouting_array]
            fit = fitness[scouting_array]
            obj = objective[scouting_array]
            trail = trials[scouting_array]
        
            #creating random population
            sol_new = bounds_lower + (bounds_upper-bounds_lower) .* rand(D) # -5 *(10*rand)
            new_obj = compute_objective([sol_new],f)
            new_fit = compute_fitness(new_obj)
        
            # replacing the new population
            population[scouting_array] = sol_new
            fitness[scouting_array] = new_fit[1]
            objective[scouting_array] = new_obj
            trials[scouting_array] = 0
        
        end
        return population, fitness, objective, trials  


end




Scouting (generic function with 1 method)

In [11]:
""" Creating a new solution
Creates new solution by changing one variable using partner solution.

Input
- solutions: current solution 
- population : population of solutions 
- bounds_lower: lower bounds of variables 
- bounds_upper: upper bounds of variables 

Output
- solution_new: new solution with one variable changed using the partner solutions


"""

function create_newsolution(solution, population, bounds_lower, bounds_upper)
        # select random variable to change       
        randomvar1_index = rand(1:size(solution)[1], 1)

        # select partner solution to generate new solution        
        randompartner_index = rand(1:size(population)[1], 1)

        # select random variable in partner solution to exchange with

        randompartner = population[randompartner_index, :][1]
        randomvar2_index = rand(1:length(randompartner), 1)

        # create new food location
        phi = rand()*2-1 #random number between -1 and 1     
        global solution_new = float(deepcopy(solution))
        a = solution[randomvar1_index] 
        b = randompartner[randomvar2_index]
        solution_new[randomvar1_index] = a + phi*(a - b)

        # check if lower bound is violated
        if solution_new[randomvar1_index] < bounds_lower[randomvar1_index] 
            solution_new[randomvar1_index] = bounds_lower[randomvar1_index]
        end

        # check if upper bound is violated
        if solution_new[randomvar1_index] > bounds_upper[randomvar1_index]
            solution_new[randomvar1_index] = bounds_upper[randomvar1_index]
        end
    return solution_new
end



create_newsolution (generic function with 1 method)

In [55]:
function sphere(x)
    d = length(x)
    return sum(x.^2)
end  

function ackley(x; a=20, b=0.2, c=2π)
    d = length(x)
    return -a * exp(-b*sqrt(sum(x.^2)/d)) -
        exp(sum(cos.(c .* x))/d) + a + exp(1)
end

function rosenbrock(x; a=1, b=5)
    # 2 dimensions!
    return (a-x[1])^2 + b*(x[2]-x[1]^2)^2
end

function branin((x1, x2); a=1, b=5.1/(4pi^2), c=5/pi, r=6, s=10, t=1/8pi)
    # 2 dimensions!
    return a * (x2 - b * x1^2 + c * x1 - r)^2 + s * (1 - t) * cos(x1) + s
end

function rastrigine(x; A=10)
    return length(x) * A + sum(x.^2 .- A .* cos.(2pi .* x))
end

rastrigine (generic function with 1 method)

In [13]:
using Pkg; Pkg.add("Optim")

   Updating registry at `C:\Users\kirst\.julia\registries\General`
  Resolving

LoadError: InterruptException:

In [14]:
using Optim

LoadError: ArgumentError: Package Optim not found in current path:
- Run `import Pkg; Pkg.add("Optim")` to install the Optim package.


In [15]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
population = initialize_population(D, bounds_lower, bounds_upper, S/2)
population = float(population[1])
optimize(sphere,population)
end

LoadError: UndefVarError: optimize not defined

In [16]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
limit = D * (S/2)
T= 500
ArtificialBeeColonization(D, bounds_lower, bounds_upper, S,T, limit, sphere)
end

  3.032750 seconds (8.55 M allocations: 421.005 MiB, 8.40% gc time)


([3.430208779500989e-9, -5.371843776077437e-10, 4.791696414711811e-9, -3.5595400307109653e-9], Any[Any[[-54.0, 23.453700473107848, -5.617947233599658, 9.0], [81, 91, 13, -19], [-34, -13, 15, 68], [48.84247647321747, -68.0, -16.36562055283561, -54.0], [74, 96, 61, 52], [-52.12853061286455, 77.0, -36.0, 68.0], [-92, -60, 49, 50], [12, -79, -89, 55], [-62, -60, -92, -7], [-87.0, -44.475763387439684, 8.0, 13.027269207383945], [-2, 46, 52, 48], [-21, -81, 68, 72]], Any[[-54.0, 10.612969924430686, -5.617947233599658, 7.543595150880679], [81, 91, 13, -19], [-34, -13, 15, 68], [48.84247647321747, -68.0, -16.36562055283561, -54.0], [74.0, 96.0, 55.45428209932545, 52.0], [23.030718954248343, 77.0, -36.0, 68.0], [76.32946219536001, -60.0, 49.0, 50.0], [12, -79, -89, 55], [-62.0, -45.81549714157016, -92.0, -7.0], [-87.0, -44.475763387439684, 3.192472727875937, 13.027269207383945], [-2, 46, 52, 48], [-21.0, -81.0, -2.51164766842048, 72.0]], Any[[-54.0, 6.1895782725648605, -5.617947233599658, 6.3068

In [17]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
population = initialize_population(D, bounds_lower, bounds_upper, S/2)
population = float(population[1])
optimize(ackley,population)
end

LoadError: UndefVarError: optimize not defined

In [18]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
limit = D * (S/2)
T= 500
ArtificialBeeColonization(D, bounds_lower, bounds_upper, S,T, limit, ackley)
end

  0.344198 seconds (1.52 M allocations: 87.668 MiB, 5.13% gc time)


([1.2976755464973372e-13, 5.165541199579107e-13, -1.275121150403161e-12, 3.770252544747828e-14], Any[Any[[40, -78, 95, -20], [-22, 72, -73, 43], [-45, 57, 58, 56], [-99, 34, 68, -67], [13, -30, -18, -57], [55, 2, -26, -26], [-94, 15, -65, -1], [49, -88, -48, -5], [-63, -91, 36, -84], [82, 80, -100, 90], [99, 85, -22, 30], [-69, 84, 85, -69]], Any[[40, -78, 95, -20], [-22, 72, -73, 43], [-45, 57, 58, 56], [-99, 34, 68, -67], [13.0, -30.0, -18.0, -5.040253882379034], [55, 2, -26, -26], [-94, 15, -65, -1], [49, -88, -48, -5], [-63, -91, 36, -84], [82, 80, -100, 90], [99, 85, -22, 30], [-69, 84, 85, -69]], Any[[40, -78, 95, -20], [-22, 72, -73, 43], [-45, 57, 58, 56], [-99, 34, 68, -67], [13.0, -30.0, -18.0, -5.040253882379034], [55, 2, -26, -26], [-94, 15, -65, -1], [49, -88, -48, -5], [-63, -91, 36, -84], [82, 80, -100, 90], [99, 85, -22, 30], [-69, 84, 85, -69]], Any[[40, -78, 95, -20], [-22, 72, -73, 43], [-45, 57, 58, 56], [-99, 34, 68, -67], [13.0, -30.0, -18.0, -5.040253882379034], 

In [19]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
population = initialize_population(D, bounds_lower, bounds_upper, S/2)
population = float(population[1])
optimize(rastrigine,population)
end

LoadError: UndefVarError: optimize not defined

In [20]:
@time begin
S = 24
D = 4
bounds_lower = [-100,-100, -100,-100];
bounds_upper = [100,100, 100,100];
limit = D * (S/2)
T= 50
ArtificialBeeColonization(D, bounds_lower, bounds_upper, S,T, limit, rastrigine)
end

  0.246558 seconds (1.09 M allocations: 55.010 MiB, 4.08% gc time)


([0.49930723182875036, 0.4991251231894678, -0.5069151121369301, 0.5032996747752302], Any[Any[[46, -79, 100, -32], [-3.086872714661837, -66.0, 72.0, -61.0], [-45, -82, -46, -14], [-84.0, -12.289138737676712, 46.0, 18.0], [69.0, -13.045051673373813, 98.0, -27.470085730500802], [-68, 98, -67, -55], [-78.0, -39.349240229150084, -89.0, -44.0], [-81, 19, 4, 56], [-56.0, -4.645484724942193, 39.0, -26.0], [-54.0, 82.0, -22.150963870931896, 57.8538850987851], [35, -53, -22, -12], [48, 30, -41, 41]], Any[[46, -79, 100, -32], [-3.086872714661837, 9.431176699961213, 72.0, -16.084774403890435], [-22.609768273694904, -82.0, -46.0, -14.0], [-84.0, -12.289138737676712, 46.0, 18.0], [68.99606361816366, -13.045051673373813, 98.0, -27.470085730500802], [35.36940285003065, 98.0, -67.0, -55.0], [-78.0, -39.349240229150084, -89.0, -36.33797001052216], [-81.0, 19.0, 4.0, -0.342963939187654], [-56.0, -4.645484724942193, 39.0, -26.0], [-54.0, 82.0, 20.491893933150372, -44.55329331860548], [35.0, -53.0, 11.9419

In [21]:
@time begin
S = 24
D = 2
bounds_lower = [-100,-100];
bounds_upper = [100,100];
population = initialize_population(D, bounds_lower, bounds_upper, S/2)
population = float(population[1])
optimize(rosenbrock,population)
end

LoadError: UndefVarError: optimize not defined

In [22]:
@time begin
S = 24
D = 2
bounds_lower = [-5,-5];
bounds_upper = [5,5];
limit = D * (S/2)
T= 50
ArtificialBeeColonization(D, bounds_lower, bounds_upper, S,T, limit, rosenbrock)
end

  0.075410 seconds (145.20 k allocations: 7.753 MiB)


([0.9926501694042553, 0.9820415167392611], Any[Any[[3.0, -0.9491830973545583], [1.0, -2.6301809581788564], [1, 5], [0.8253249820159727, 1.0], [-4.0, 5.0], [4, -2], [-1, 1], [1.086404540261745, 3.0], [-1.5931841078076547, -5.0], [-1.0, 0.5477349106694475], [2.7353295888663567, 4.0], [2.0, 0.022117214943186525]], Any[[3.0, -0.38181735426810326], [1.0, -2.6301809581788564], [1.0, 3.5500366703013246], [0.9800006242469805, 1.0], [-4.0, 5.0], [3.5519429802831795, -2.0], [-0.9425192939261633, 1.0], [1.0643675507057653, 1.025277660632284], [-1.5931841078076547, -5.0], [-0.7218427669712653, 0.5477349106694475], [2.7353295888663567, 4.0], [2.0, 0.022117214943186525]], Any[[2.0878844901986837, -0.38181735426810326], [1.0, 2.4289952838454685], [1.0, 0.06293192514059842], [0.9800006242469805, 1.0], [-4.0, 5.0], [2.6731367593336897, 0.6231088715648232], [-0.9425192939261633, 1.0], [1.0643675507057653, 1.025277660632284], [-1.4539762797502063, -5.0], [-0.7218427669712653, 0.5477349106694475], [2.7353

In [23]:
@time begin
S = 24
D = 2
bounds_lower = [-100,-100];
bounds_upper = [100,100];
population = initialize_population(D, bounds_lower, bounds_upper, S/2)
population = float(population[1])
optimize(branin,population)
end

LoadError: UndefVarError: optimize not defined

In [24]:
@time begin
S = 24
D = 2
bounds_lower = [-5,-5];
bounds_upper = [5,5];
limit = D * (S/2)
T= 50
ArtificialBeeColonization(D, bounds_lower, bounds_upper, S,T, limit, branin)
end

  0.095236 seconds (306.08 k allocations: 16.807 MiB)


([3.1416885259269334, 2.275250625840672], Any[Any[[2.2698900829334625, -4.0], [3.0, -1.5340033824024548], [3, -4], [-2.0, 2.4601567039720322], [-5.0, 2.9122281077412975], [4, -2], [3.0, 2.760013665662253], [3.6571513237334234, 1.0], [5.0, 2.0571568390995094], [5.0, 1.4298733285506433], [4.0, 1.5303340937171042], [4.74686400716513, 2.0]], Any[[2.2698900829334625, -4.0], [3.0, -1.5340033824024548], [3, -4], [-1.4102476244513902, 2.4601567039720322], [-5.0, 3.110885646241522], [4.0, 1.450184372210253], [3.0, 2.389878199970906], [3.6571513237334234, 2.0434381895149656], [5.0, 1.6576942543218605], [3.4805444290656515, 1.4298733285506433], [4.0, 1.5303340937171042], [4.74686400716513, 2.0]], Any[[2.2698900829334625, -0.018903764947555413], [3.0, -1.5340033824024548], [3.0, -3.1950859421219873], [-1.4102476244513902, 2.4601567039720322], [-5.0, 3.110885646241522], [3.6065731707542232, 1.450184372210253], [3.0190091850419614, 2.389878199970906], [3.1994664775742483, 2.0434381895149656], [5.0, 